# Notebook for unsupervised ML for conservation laws

In [ ]:
# Code taken and adapted from: https://github.com/KindXiaoming/aipoincare/blob/master/backend/training.py
# Dataset from: https://github.com/KindXiaoming/aipoincare/blob/master/backend/data/oned_harmonic
# ChatGPT assistance: https://chatgpt.com/share/6882476e-0d50-8011-b66e-4d9e376e7696

# In case some packages in correct version should be installed, there is a requirements.txt in working_tree, and a code cell below for "!pip install ..."

In [2]:
# ------------------------------------------------------------
# 1a. Mount Google Drive (do this once per Colab session)
# ------------------------------------------------------------
import pickle
from google.colab import drive
drive.mount('/content/drive/')
dir_path = '/content/drive/MyDrive/AI-in-physics_AJP_2025/Conservation-laws/' # needs to be adapted to specific context

Mounted at /content/drive/


In [ ]:
# ------------------------------------------------------------
# 1b. If you are on a local machine, set dir to current working tree
# ------------------------------------------------------------
import pickle
dir_path = '../Conservation-laws/' # needs to be adapted to specific context

In [3]:
# --------------------- Import necessary libraries ---------------------
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
from sklearn.decomposition import PCA
import torch.optim as optim
import copy

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

In [1]:
# --------------------- Set hyperparameters and data config ---------------------

# Whether to remove PCA components below a noise threshold or keep them (normalize instead)
prepcaremove = 'no'  # 'no' = normalize all components, 'yes' = discard low-variance ones
# Variance threshold for PCA dimension filtering
noise_threshold = 0.001
# Hidden layer widths for the neural network; first and last layer are added later
nn_widths = [256, 256]  # Two hidden layers: 64 units and 32 units
hidden_depth = len(nn_widths)
# Negative slope for LeakyReLU activation function
slope = 0.1
# List of noise magnitudes (sigmal) to test during training
sigmals = [1e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1.0, 10.0]
# Optimizer type: 'Adam' or 'SGD'
opt = 'Adam'
# Learning rate for optimizer
lr = 0.001
# Mini-batch size for training
batch_size = 64
# Number of training iterations (epochs)
epoch = 5000
# ERD parameter `a` for computing effective dimensionality
a = 2
# Path to the input data file (must be a .txt file of shape [n_samples, n_features])
model = 'harmonic_1d.txt'
# Number of random walks to simulate for post-training variance analysis
n_walk = 600
# Frequency of logging during training
log = 200

In [4]:
# --------------------- Load and preprocess the data ---------------------

# Load dataset as NumPy array
xs = np.loadtxt(os.path.join(dir_path, model).replace("\\", "/"))
n_train = xs.shape[0]       # Number of samples
input_dim = xs.shape[1]     # Dimensionality of input features

# Step 1: Normalize each input feature to zero mean and unit variance
xs = (xs - np.mean(xs, axis=0)[np.newaxis, :]) / np.std(xs, axis=0)[np.newaxis, :]

# Step 2: Apply PCA (rotation only, no scaling) to decorrelate dimensions
pca = PCA()
xs = pca.fit_transform(xs)

# Step 3: Remove or normalize dimensions based on explained variance
remove_dim = 0
if prepcaremove == "no":
    # Keep all dimensions, normalize each one by variance + threshold
    xs = xs / (np.std(xs, axis=0)[np.newaxis, :] + noise_threshold)
else:
    # Keep only dimensions where variance > threshold
    input_dim_orig = input_dim
    input_dim = np.sum(pca.explained_variance_ratio_ > noise_threshold)
    remove_dim = input_dim_orig - input_dim
    xs = xs[:, :input_dim]
    xs = xs / (np.std(xs, axis=0)[np.newaxis, :])

# Add input/output layers to the neural network layer list
nn_widths.insert(0, input_dim)
nn_widths.append(input_dim)

In [5]:
# --------------------- Training loop across different noise levels ---------------------
exps = []   # To store explained variance ratios
losses = [] # To record loss values during training

for sigmal in sigmals:
    # Define a new neural network (as a list of layers) for each noise level
    linears = [nn.Linear(nn_widths[i], nn_widths[i + 1]) for i in range(hidden_depth + 1)]
    parameters = [p for layer in linears for p in layer.parameters()]  # Collect all trainable parameters

    # Define loss function
    criterion = nn.MSELoss()

    # Choose optimizer
    optimizer = optim.Adam(parameters, lr=lr) if opt == "Adam" else optim.SGD(parameters, lr=lr)

    print(f"sigmal = {sigmal}")

    # --------------------- Training the denoising network ---------------------
    for j in range(epoch):
        # Randomly sample a batch of training data
        choices = np.random.choice(n_train, batch_size)
        perturb = torch.normal(0, sigmal, size=(batch_size, input_dim))  # Add Gaussian noise
        inputs0 = torch.tensor(xs[choices], dtype=torch.float) + perturb  # Noisy input

        # Forward pass through each layer using LeakyReLU
        x = inputs0
        act = nn.LeakyReLU(slope)
        for i in range(hidden_depth):
            x = act(linears[i](x))
        outputs = linears[hidden_depth](x)  # Final layer (no activation)

        # Training objective: recover the *negative* of the noise added
        loss = criterion(outputs, -perturb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(float(loss.data))

        if j % log == 0:
            print(f"Epoch {j} | Loss: {loss:.4f}")

    # --------------------- Perform stochastic walks using trained network ---------------------
    x0 = copy.deepcopy(xs[int(n_train / 2)])  # Start from a central data point
    x0 = x0[np.newaxis, :]
    x0 = x0 + np.random.randn(n_walk, input_dim) * sigmal  # Add noise

    # Pass walk samples through trained denoising network
    x = torch.tensor(x0, dtype=torch.float)
    for i in range(hidden_depth):
        x = act(linears[i](x))
    x = linears[hidden_depth](x)
    x0 = x0 + x.detach().numpy()  # Apply learned correction

    # --------------------- Compute explained variance ratio via PCA ---------------------
    pca = PCA()
    pca.fit(x0)
    svs = pca.singular_values_
    exp_ratio = svs**2 / np.sum(svs**2)
    exps.append(exp_ratio)

# Stack results into a NumPy array
exps = np.array(exps)

sigmal = 0.001
Epoch 0 | Loss: 0.0180
Epoch 200 | Loss: 0.0000
Epoch 400 | Loss: 0.0000
Epoch 600 | Loss: 0.0000
Epoch 800 | Loss: 0.0000
Epoch 1000 | Loss: 0.0000
Epoch 1200 | Loss: 0.0000
Epoch 1400 | Loss: 0.0000
Epoch 1600 | Loss: 0.0000
Epoch 1800 | Loss: 0.0000
Epoch 2000 | Loss: 0.0000
Epoch 2200 | Loss: 0.0000
Epoch 2400 | Loss: 0.0000
Epoch 2600 | Loss: 0.0000
Epoch 2800 | Loss: 0.0000
Epoch 3000 | Loss: 0.0000
Epoch 3200 | Loss: 0.0000
Epoch 3400 | Loss: 0.0000
Epoch 3600 | Loss: 0.0000
Epoch 3800 | Loss: 0.0000
Epoch 4000 | Loss: 0.0000
Epoch 4200 | Loss: 0.0000
Epoch 4400 | Loss: 0.0000
Epoch 4600 | Loss: 0.0000
Epoch 4800 | Loss: 0.0000
sigmal = 0.01
Epoch 0 | Loss: 0.0053
Epoch 200 | Loss: 0.0001
Epoch 400 | Loss: 0.0002
Epoch 600 | Loss: 0.0002
Epoch 800 | Loss: 0.0001
Epoch 1000 | Loss: 0.0002
Epoch 1200 | Loss: 0.0001
Epoch 1400 | Loss: 0.0001
Epoch 1600 | Loss: 0.0001
Epoch 1800 | Loss: 0.0001
Epoch 2000 | Loss: 0.0001
Epoch 2200 | Loss: 0.0001
Epoch 2400 | Loss: 0.00

In [6]:
# --------------------- Define helper function to compute effective rank (ERD) ---------------------
def f(x, a=2):
    n = x.shape[1]
    mask = x < 1 / (a * n)
    return np.sum(np.cos(np.pi / 2 * n * a * x) * mask, axis=1)

# --------------------- Plot explained variance and effective dimensionality ---------------------
plt.figure(figsize=(7, 5))

# Plot each PCA component’s explained variance
for i in range(input_dim):
    plt.plot(sigmals, exps[:, i], marker="o")

plt.xscale('log')
plt.xlabel(r"$L$", fontsize=25)
plt.ylabel("Explained Ratio", fontsize=25)

# Plot effective dimensionality (ERD) on second y-axis
ax2 = plt.gca().twinx()
ax2.plot(sigmals, f(exps, a=2), marker="o", color="red", linewidth=5, ls="--", markersize=15)
plt.ylabel(r"$m_{eff}$", fontsize=25, color="red")

# Save the plot to disk
plt.savefig(os.path.join(dir_path, 'ERD.png').replace("\\", "/"), bbox_inches="tight")
plt.clf()

/tmp/ipython-input-6-4039673660.py:25: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  plt.clf()


<Figure size 700x500 with 0 Axes>

In [ ]:
""" uncomment, if packages should be installed:
!pip install Bottleneck==1.4.2 Cython==3.0.12 Farama-Notifications==0.0.4 GDAL==3.8.4 GitPython==3.1.44 Mako==1.1.3 MarkupSafe==3.0.2 PyDrive2==1.21.3 PyDrive==1.3.1 PyGObject==3.42.0 PyJWT==2.10.1 PyOpenGL==3.1.9 PySocks==1.7.1 PyYAML==6.0.2 SecretStorage==3.3.3 Send2Trash==1.8.3 absl-py==1.4.0 accelerate==1.8.1 aiofiles==24.1.0 aiohappyeyeballs==2.6.1 aiohttp==3.11.15 aiosignal==1.3.2 alabaster==1.0.0 albucore==0.0.24 albumentations==2.0.8 ale-py==0.11.1 altair==5.5.0 annotated-types==0.7.0 antlr4-python3-runtime==4.9.3 anyio==4.9.0 argon2-cffi-bindings==21.2.0 argon2-cffi==25.1.0 array-record==0.7.2 arviz==0.21.0 astropy-iers-data==0.2025.6.30.0.39.40 astropy==7.1.0 astunparse==1.6.3 atpublic==5.1 attrs==25.3.0 audioread==3.0.1 autocommand==2.2.2 autograd==1.8.0 babel==2.17.0 backcall==0.2.0 backports.tarfile==1.2.0 beautifulsoup4==4.13.4 betterproto==2.0.0b6 bigframes==2.8.0 bigquery-magics==0.9.0 bleach==6.2.0 blinker==1.9.0 blis==1.3.0 blobfile==3.0.0 blosc2==3.5.1 bokeh==3.7.3 bqplot==0.12.45 branca==0.8.1 build==1.2.2.post1 cachecontrol==0.14.3 cachetools==5.5.2 catalogue==2.0.10 certifi==2025.6.15 cffi==1.17.1 chardet==5.2.0 charset-normalizer==3.4.2 chex==0.1.89 clarabel==0.11.1 click==8.2.1 cloudpathlib==0.21.1 cloudpickle==3.1.1 cmake==3.31.6 cmdstanpy==1.2.5 colorcet==3.1.0 colorlover==0.3.0 colour==0.1.5 community==1.0.0b1 confection==0.1.5 cons==0.4.6 contourpy==1.3.2 cramjam==2.10.0 cryptography==43.0.3 cuda-python==12.6.2.post1 cudf-cu12==25.2.1 cudf-polars-cu12==25.2.2 cufflinks==0.17.3 cuml-cu12==25.2.1 cupy-cuda12x==13.3.0 curl-cffi==0.11.4 cuvs-cu12==25.2.1 cvxopt==1.3.2 cvxpy==1.6.6 cycler==0.12.1 cyipopt==1.5.0 cymem==2.0.11 dask-cuda==25.2.0 dask-cudf-cu12==25.2.2 dask-expr==1.1.21 dask==2024.12.1 dataproc-spark-connect==0.8.1 datascience==0.17.6 datasets==2.14.4 db-dtypes==1.4.3 dbus-python==1.2.18 debugpy==1.8.0 decorator==4.4.2 defusedxml==0.7.1 diffusers==0.34.0 dill==0.3.7 distributed-ucxx-cu12==0.42.0 distributed==2024.12.1 distro==1.9.0 dlib==19.24.6 dm-tree==0.1.9 docstring-parser==0.16 docutils==0.21.2 dopamine-rl==4.1.2 duckdb==1.2.2 earthengine-api==1.5.22 easydict==1.13 editdistance==0.8.1 eerepr==0.1.2 einops==0.8.1 en-core-web-sm==3.8.0 entrypoints==0.4 et-xmlfile==2.0.0 etils==1.12.2 etuples==0.3.9 fastai==2.7.19 fastapi==0.115.14 fastcore==1.7.29 fastdownload==0.0.7 fastjsonschema==2.21.1 fastprogress==1.0.3 fastrlock==0.8.3 ffmpy==0.6.0 filelock==3.18.0 firebase-admin==6.9.0 flask==3.1.1 flatbuffers==25.2.10 flax==0.10.6 folium==0.19.7 fonttools==4.58.4 frozendict==2.4.6 frozenlist==1.7.0 fsspec==2025.3.2 future==1.0.0 gast==0.6.0 gcsfs==2025.3.2 gdown==5.2.0 geemap==0.35.3 geocoder==1.38.1 geographiclib==2.0 geopandas==1.0.1 geopy==2.4.1 gin-config==0.5.0 gitdb==4.0.12 glob2==0.7 google-ai-generativelanguage==0.6.15 google-api-core==2.25.1 google-api-python-client==2.174.0 google-auth-httplib2==0.2.0 google-auth-oauthlib==1.2.2 google-auth==2.38.0 google-cloud-aiplatform==1.101.0 google-cloud-bigquery-connection==1.18.3 google-cloud-bigquery-storage==2.32.0 google-cloud-bigquery==3.34.0 google-cloud-core==2.4.3 google-cloud-dataproc==5.21.0 google-cloud-datastore==2.21.0 google-cloud-firestore==2.21.0 google-cloud-functions==1.20.4 google-cloud-iam==2.19.1 google-cloud-language==2.17.2 google-cloud-resource-manager==1.14.2 google-cloud-spanner==3.55.0 google-cloud-storage==2.19.0 google-cloud-translate==3.21.1 google-colab==1.0.0 google-crc32c==1.7.1 google-genai==1.24.0 google-generativeai==0.8.5 google-pasta==0.2.0 google-resumable-media==2.7.2 google==2.0.3 googleapis-common-protos==1.70.0 googledrivedownloader==1.1.0 gradio-client==1.10.1 gradio==5.31.0 graphviz==0.21 greenlet==3.2.3 groovy==0.1.2 grpc-google-iam-v1==0.14.2 grpc-interceptor==0.15.4 grpcio-status==1.71.2 grpcio==1.73.1 grpclib==0.4.8 gspread-dataframe==4.0.0 gspread==6.2.1 gym-notices==0.0.8 gym==0.25.2 gymnasium==1.2.0 h11==0.16.0 h2==4.2.0 h5netcdf==1.6.3 h5py==3.14.0 hdbscan==0.8.40 hf-transfer==0.1.9 hf-xet==1.1.5 highspy==1.11.0 holidays==0.75 holoviews==1.21.0 hpack==4.1.0 html5lib==1.1 httpcore==1.0.9 httpimport==1.4.1 httplib2==0.22.0 httpx==0.28.1 huggingface-hub==0.33.2 humanize==4.12.3 hyperframe==6.1.0 hyperopt==0.2.7 ibis-framework==9.5.0 idna==3.10 imageio-ffmpeg==0.6.0 imageio==2.37.0 imagesize==1.4.1 imbalanced-learn==0.13.0 immutabledict==4.2.1 importlib-metadata==8.7.0 importlib-resources==6.5.2 imutils==0.5.4 inflect==7.5.0 iniconfig==2.1.0 intel-cmplr-lib-ur==2025.2.0 intel-openmp==2025.2.0 ipyevents==2.0.2 ipyfilechooser==0.6.0 ipykernel==6.17.1 ipyleaflet==0.20.0 ipyparallel==8.8.0 ipython-genutils==0.2.0 ipython-sql==0.5.0 ipython==7.34.0 ipytree==0.2.2 ipywidgets==7.7.1 itsdangerous==2.2.0 jaraco-functools==4.2.1 jaraco.classes==3.4.0 jaraco.collections==5.1.0 jaraco.context==6.0.1 jaraco.functools==4.0.1 jaraco.text==3.12.1 jax-cuda12-pjrt==0.5.1 jax-cuda12-plugin==0.5.1 jax==0.5.2 jaxlib==0.5.1 jeepney==0.9.0 jieba==0.42.1 jinja2==3.1.6 jiter==0.10.0 joblib==1.5.1 jsonpatch==1.33 jsonpickle==4.1.1 jsonpointer==3.0.0 jsonschema-specifications==2025.4.1 jsonschema==4.24.0 jupyter-client==6.1.12 jupyter-console==6.1.0 jupyter-core==5.8.1 jupyter-kernel-gateway==2.5.2 jupyter-leaflet==0.20.0 jupyter-server==1.16.0 jupyterlab-pygments==0.3.0 jupyterlab-widgets==3.0.15 jupytext==1.17.2 kaggle==1.7.4.5 kagglehub==0.3.12 keras-hub==0.18.1 keras-nlp==0.18.1 keras==3.8.0 keyring==25.6.0 keyrings.google-artifactregistry-auth==1.1.2 kiwisolver==1.4.8 langchain-core==0.3.67 langchain-text-splitters==0.3.8 langchain==0.3.26 langcodes==3.5.0 langsmith==0.4.4 language-data==1.3.0 launchpadlib==1.10.16 lazr.restfulclient==0.14.4 lazr.uri==1.0.6 lazy-loader==0.4 libclang==18.1.1 libcudf-cu12==25.2.1 libcugraph-cu12==25.2.0 libcuml-cu12==25.2.1 libcuvs-cu12==25.2.1 libkvikio-cu12==25.2.1 libpysal==4.13.0 libraft-cu12==25.2.0 librosa==0.11.0 libucx-cu12==1.18.1 libucxx-cu12==0.42.0 lightgbm==4.5.0 linkify-it-py==2.0.3 llvmlite==0.43.0 locket==1.0.0 logical-unification==0.4.6 lxml==5.4.0 marisa-trie==1.2.1 markdown-it-py==3.0.0 markdown==3.8.2 matplotlib-inline==0.1.7 matplotlib-venn==1.1.2 matplotlib==3.10.0 mdit-py-plugins==0.4.2 mdurl==0.1.2 miniKanren==1.0.3 missingno==0.5.2 mistune==3.1.3 mizani==0.13.5 mkl==2025.0.1 ml-dtypes==0.4.1 mlxtend==0.23.4 more-itertools==10.7.0 moviepy==1.0.3 mpmath==1.3.0 msgpack==1.1.1 multidict==6.6.3 multipledispatch==1.0.0 multiprocess==0.70.15 multitasking==0.0.11 murmurhash==1.0.13 music21==9.3.0 namex==0.1.0 narwhals==1.45.0 natsort==8.4.0 nbclassic==1.3.1 nbclient==0.10.2 nbconvert==7.16.6 nbformat==5.10.4 ndindex==1.10.0 nest-asyncio==1.6.0 networkx==3.5 nibabel==5.3.2 nltk==3.9.1 notebook-shim==0.2.4 notebook==6.5.7 numba-cuda==0.2.0 numba==0.60.0 numexpr==2.11.0 numpy==2.0.2 nvidia-cublas-cu12==12.5.3.2 nvidia-cuda-cupti-cu12==12.5.82 nvidia-cuda-nvcc-cu12==12.5.82 nvidia-cuda-nvrtc-cu12==12.5.82 nvidia-cuda-runtime-cu12==12.5.82 nvidia-cudnn-cu12==9.3.0.75 nvidia-cufft-cu12==11.2.3.61 nvidia-curand-cu12==10.3.6.82 nvidia-cusolver-cu12==11.6.3.83 nvidia-cusparse-cu12==12.5.1.3 nvidia-cusparselt-cu12==0.6.2 nvidia-ml-py==12.575.51 nvidia-nccl-cu12==2.21.5 nvidia-nvcomp-cu12==4.2.0.11 nvidia-nvjitlink-cu12==12.5.82 nvidia-nvtx-cu12==12.4.127 nvtx==0.2.12 nx-cugraph-cu12==25.2.0 oauth2client==4.1.3 oauthlib==3.3.1 omegaconf==2.3.0 openai==1.93.0 opencv-contrib-python==4.11.0.86 opencv-python-headless==4.11.0.86 opencv-python==4.11.0.86 openpyxl==3.1.5 opt-einsum==3.4.0 optax==0.2.5 optree==0.16.0 orbax-checkpoint==0.11.16 orjson==3.10.18 osqp==1.0.4 packaging==24.2 pandas-datareader==0.10.0 pandas-gbq==0.29.1 pandas-stubs==2.2.2.240909 pandas==2.2.2 pandocfilters==1.5.1 panel==1.7.2 param==2.2.1 parso==0.8.4 parsy==2.1 partd==1.4.2 pathlib==1.0.1 patsy==1.0.1 peewee==3.18.1 peft==0.15.2 pexpect==4.9.0 pickleshare==0.7.5 pillow==11.2.1 pip==24.1.2 platformdirs==4.3.8 plotly==5.24.1 plotnine==0.14.6 pluggy==1.6.0 ply==3.11 polars==1.21.0 pooch==1.8.2 portpicker==1.5.2 preshed==3.0.10 prettytable==3.16.0 proglog==0.1.12 progressbar2==4.5.0 prometheus-client==0.22.1 promise==2.3 prompt-toolkit==3.0.51 propcache==0.3.2 prophet==1.1.7 proto-plus==1.26.1 protobuf==5.29.5 psutil==5.9.5 psycopg2==2.9.10 ptyprocess==0.7.0 py-cpuinfo==9.0.0 py4j==0.10.9.7 pyOpenSSL==24.2.1 pyarrow==18.1.0 pyasn1-modules==0.4.2 pyasn1==0.6.1 pycairo==1.28.0 pycocotools==2.0.10 pycparser==2.22 pycryptodomex==3.23.0 pydantic-core==2.33.2 pydantic==2.11.7 pydata-google-auth==1.9.1 pydot==3.0.4 pydotplus==2.0.2 pydub==0.25.1 pyerfa==2.0.1.5 pygame==2.6.1 pygit2==1.18.0 pygments==2.19.2 pylibcudf-cu12==25.2.1 pylibcugraph-cu12==25.2.0 pylibraft-cu12==25.2.0 pymc==5.23.0 pymystem3==0.2.0 pynndescent==0.5.13 pynvjitlink-cu12==0.7.0 pynvml==12.0.0 pyogrio==0.11.0 pyomo==6.9.2 pyparsing==3.2.3 pyperclip==1.9.0 pyproj==3.7.1 pyproject-hooks==1.2.0 pyshp==2.3.1 pyspark==3.5.1 pytensor==2.31.5 pytest==8.3.5 python-apt==0.0.0 python-box==7.3.2 python-dateutil==2.9.0.post0 python-louvain==0.16 python-multipart==0.0.20 python-slugify==8.0.4 python-snappy==0.7.3 python-utils==3.9.1 pytz==2025.2 pyviz-comms==3.0.6 pywavelets==1.8.0 pyzmq==24.0.1 raft-dask-cu12==25.2.0 rapids-dask-dependency==25.2.0 ratelim==0.1.6 referencing==0.36.2 regex==2024.11.6 requests-oauthlib==2.0.0 requests-toolbelt==1.0.0 requests==2.32.3 requirements-parser==0.9.0 rich==13.9.4 rmm-cu12==25.2.0 roman-numerals-py==3.1.0 rpds-py==0.26.0 rpy2==3.5.17 rsa==4.9.1 ruff==0.12.1 safehttpx==0.1.6 safetensors==0.5.3 scikit-image==0.25.2 scikit-learn==1.6.1 scipy==1.15.3 scooby==0.10.1 scs==3.2.7.post2 seaborn==0.13.2 semantic-version==2.10.0 sentence-transformers==4.1.0 sentencepiece==0.2.0 sentry-sdk==2.32.0 setuptools==75.2.0 shap==0.48.0 shapely==2.1.1 shellingham==1.5.4 simple-parsing==0.1.7 simplejson==3.20.1 simsimd==6.4.9 six==1.17.0 sklearn-compat==0.1.3 sklearn-pandas==2.2.0 slicer==0.0.8 smart-open==7.3.0 smmap==5.0.2 sniffio==1.3.1 snowballstemmer==3.0.1 sortedcontainers==2.4.0 soundfile==0.13.1 soupsieve==2.7 soxr==0.5.0.post1 spacy-legacy==3.0.12 spacy-loggers==1.0.5 spacy==3.8.7 spanner-graph-notebook==1.1.7 sphinx==8.2.3 sphinxcontrib-applehelp==2.0.0 sphinxcontrib-devhelp==2.0.0 sphinxcontrib-htmlhelp==2.1.0 sphinxcontrib-jsmath==1.0.1 sphinxcontrib-qthelp==2.0.0 sphinxcontrib-serializinghtml==2.0.0 sqlalchemy==2.0.41 sqlglot==25.20.2 sqlparse==0.5.3 srsly==2.5.1 stanio==0.5.1 starlette==0.46.2 statsmodels==0.14.4 stringzilla==3.12.5 stumpy==1.13.0 sympy==1.13.1 tables==3.10.2 tabulate==0.9.0 tbb==2022.2.0 tblib==3.1.0 tcmlib==1.4.0 tenacity==8.5.0 tensorboard-data-server==0.7.2 tensorboard==2.18.0 tensorflow-datasets==4.9.9 tensorflow-decision-forests==1.11.0 tensorflow-hub==0.16.1 tensorflow-io-gcs-filesystem==0.37.1 tensorflow-metadata==1.17.2 tensorflow-probability==0.25.0 tensorflow-text==2.18.1 tensorflow==2.18.0 tensorstore==0.1.74 termcolor==3.1.0 terminado==0.18.1 text-unidecode==1.3 textblob==0.19.0 tf-keras==2.18.0 tf-slim==1.1.0 thinc==8.3.6 threadpoolctl==3.6.0 tifffile==2025.6.11 tiktoken==0.9.0 timm==1.0.16 tinycss2==1.4.0 tokenizers==0.21.2 toml==0.10.2 tomli==2.0.1 tomlkit==0.13.3 toolz==0.12.1 torch==2.6.0+cu124 torchao==0.10.0 torchaudio==2.6.0+cu124 torchdata==0.11.0 torchsummary==1.5.1 torchtune==0.6.1 torchvision==0.21.0+cu124 tornado==6.4.2 tqdm==4.67.1 traitlets==5.7.1 traittypes==0.2.1 transformers==4.53.0 treelite==4.4.1 treescope==0.1.9 triton==3.2.0 tsfresh==0.21.0 tweepy==4.15.0 typeguard==4.4.4 typer==0.16.0 types-pytz==2025.2.0.20250516 types-setuptools==80.9.0.20250529 typing-extensions==4.14.0 typing-inspection==0.4.1 tzdata==2025.2 tzlocal==5.3.1 uc-micro-py==1.0.3 ucx-py-cu12==0.42.0 ucxx-cu12==0.42.0 umap-learn==0.5.9.post2 umf==0.11.0 uritemplate==4.2.0 urllib3==2.4.0 uvicorn==0.35.0 vega-datasets==0.9.0 wadllib==1.3.6 wandb==0.21.0 wasabi==1.1.3 wcwidth==0.2.13 weasel==0.4.1 webcolors==24.11.1 webencodings==0.5.1 websocket-client==1.8.0 websockets==15.0.1 werkzeug==3.1.3 wheel==0.45.1 widgetsnbextension==3.6.10 wordcloud==1.9.4 wrapt==1.17.2 wurlitzer==3.1.1 xarray-einstats==0.9.1 xarray==2025.3.1 xgboost==2.1.4 xlrd==2.0.2 xxhash==3.5.0 xyzservices==2025.4.0 yarl==1.20.1 ydf==0.12.0 yellowbrick==1.5 yfinance==0.2.64 zict==3.0.0 zipp==3.23.0 zstandard==0.23.0
"""

!pip install Bottleneck==1.4.2 Cython==3.0.12 Farama-Notifications==0.0.4 GDAL==3.8.4 GitPython==3.1.44 Mako==1.1.3 MarkupSafe==3.0.2 PyDrive2==1.21.3 PyDrive==1.3.1 PyGObject==3.42.0 PyJWT==2.10.1 PyOpenGL==3.1.9 PySocks==1.7.1 PyYAML==6.0.2 SecretStorage==3.3.3 Send2Trash==1.8.3 absl-py==1.4.0 accelerate==1.8.1 aiofiles==24.1.0 aiohappyeyeballs==2.6.1 aiohttp==3.11.15 aiosignal==1.3.2 alabaster==1.0.0 albucore==0.0.24 albumentations==2.0.8 ale-py==0.11.1 altair==5.5.0 annotated-types==0.7.0 antlr4-python3-runtime==4.9.3 anyio==4.9.0 argon2-cffi-bindings==21.2.0 argon2-cffi==25.1.0 array-record==0.7.2 arviz==0.21.0 astropy-iers-data==0.2025.6.30.0.39.40 astropy==7.1.0 astunparse==1.6.3 atpublic==5.1 attrs==25.3.0 audioread==3.0.1 autocommand==2.2.2 autograd==1.8.0 babel==2.17.0 backcall==0.2.0 backports.tarfile==1.2.0 beautifulsoup4==4.13.4 betterproto==2.0.0b6 bigframes==2.8.0 bigquery-magics==0.9.0 bleach==6.2.0 blinker==1.9.0 blis==1.3.0 blobfile==3.0.0 blosc2==3.5.1 bokeh==3.7.3 b